<a href="https://colab.research.google.com/github/dany444/one/blob/main/luchay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!apt update
#!apt install -y libreoffice
#!pip install pdf2docx python-docx ipywidgets fitz
#!pip install frontend
#!pip uninstall pymupdf -y
#!pip install pymupdf
import fitz  # PyMuPDF
import fitz  # PyMuPDF
import ipywidgets as widgets
from IPython.display import display
import os
import uuid
from pdf2docx import Converter
from docx import Document
from docx.shared import Inches
import shutil
from io import BytesIO
from PIL import Image

class PDFEditor:
    def __init__(self, pdf_url=None):
        self.pdf_path = ""
        self.text_blocks = []  # Список текстовых блоков с координатами
        self.text_coords = []  # Список координат для текста
        self.raw_text = ""
        self.temp_pdf_path = None
        self.word_file_path = None
        self.images = []  # Список изображений для вставки в Word

        # Загрузка PDF
        self.upload_button = widgets.FileUpload(accept='.pdf', multiple=False)
        self.upload_button.observe(self.load_pdf, names='value')

        # Кнопки
        self.edit_button = widgets.Button(description="Внести правки", disabled=True)
        self.edit_button.on_click(self.extract_text)

        self.save_button = widgets.Button(description="Сохранить изменения", disabled=True)
        self.save_button.on_click(self.save_pdf)

        self.refresh_button = widgets.Button(description="Обновить", disabled=False)
        self.refresh_button.on_click(self.refresh_text)

        # Текстовые поля
        self.encoded_text = widgets.Textarea(layout=widgets.Layout(width='50%', height='500px'))
        self.decoded_text = widgets.HTML(layout=widgets.Layout(width='50%', height='500px'))

        # Поиск
        self.search_input = widgets.Text(placeholder="Введите закодированный текст для поиска", layout=widgets.Layout(width='50%'))
        self.search_button = widgets.Button(description="Найти", disabled=False)
        self.search_button.on_click(self.search_and_highlight)

        # Отображение
        display(self.upload_button, self.edit_button, self.save_button, self.refresh_button)
        display(self.search_input, self.search_button)
        display(widgets.HBox([self.encoded_text, self.decoded_text]))

        if pdf_url:
            self.download_pdf(pdf_url)

    def download_pdf(self, url):
        filename = "/content/final_pdf_with_text (8) (1)_removed.pdf"
        os.system(f"wget -O {filename} {url}")
        self.pdf_path = filename
        self.edit_button.disabled = False

    def load_pdf(self, change):
        if self.upload_button.value:
            self.pdf_path = list(self.upload_button.value.keys())[0]
            self.edit_button.disabled = False

    def extract_text(self, event):
        if not self.pdf_path:
            return

        # Извлекаем текст и координаты из оригинального PDF с помощью PyMuPDF
        self.text_blocks = []
        self.text_coords = []
        self.raw_text = ""
        self.images = []  # Очистим список изображений

        doc = fitz.open(self.pdf_path)

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            blocks = page.get_text("dict")["blocks"]
            for block in blocks:
                if block["type"] == 0:  # Это текстовый блок
                    for line in block["lines"]:
                        for span in line["spans"]:
                            text = span["text"]
                            bbox = span["bbox"]  # Координаты блока текста
                            self.text_blocks.append({"text": text, "bbox": bbox})
                            self.raw_text += text + "\n"

                # Извлечение изображений
                if block["type"] == 1:  # Это изображение
                    images = page.get_images(full=True)  # Получаем список всех изображений на странице
                    for img in images:
                        xref = img[0]  # Получаем ссылку на изображение
                        base_image = doc.extract_image(xref)  # Извлекаем само изображение
                        image_bytes = base_image["image"]  # Изображение в байтах
                        self.images.append(image_bytes)

        # Отображаем извлечённый текст в кодированном и нормальном виде
        self.encoded_text.value = self.encode_text(self.raw_text)
        self.decoded_text.value = self.raw_text

        # Сохраняем путь к временной копии PDF
        self.temp_pdf_path = "/content/temp_copy.pdf"
        shutil.copy(self.pdf_path, self.temp_pdf_path)

        # Разрешаем сохранить файл
        self.save_button.disabled = False

    def encode_text(self, text):
        encoded_text = text.encode('utf-8')
        return ' '.join([f'\\x{b:02x}' for b in encoded_text])

    def search_and_highlight(self, event):
        search_term = self.search_input.value

        if search_term:
            decoded_search_term = self.decode_encoded_text(search_term)
            self.decoded_selected_text = decoded_search_term
            self.update_highlighted_text()

    def decode_encoded_text(self, encoded_text):
        encoded_bytes = bytes([int(b, 16) for b in encoded_text.split("\\x")[1:] if b])
        decoded_text = encoded_bytes.decode('utf-8', errors='ignore')
        return decoded_text

    def update_highlighted_text(self):
        if self.decoded_selected_text:
            highlighted_text = self.highlight_text_in_html(self.raw_text, self.decoded_selected_text)
            self.decoded_text.value = highlighted_text
        else:
            self.decoded_text.value = self.raw_text

    def highlight_text_in_html(self, text, selected_text):
        if selected_text:
            highlighted = f'<span style="background-color: yellow;">{selected_text}</span>'
            return text.replace(selected_text, highlighted)
        return text

    def refresh_text(self, event):
        # Обновление текста в правом окне после изменения в левом
        self.decoded_text.value = self.encoded_text.value

    def save_pdf(self, event):
        if not self.temp_pdf_path:
            return

        # Сохраняем изменения в Word
        doc = Document()

        # Добавляем изображения в документ
        for img_data in self.images:
            image_stream = BytesIO(img_data)
            image = Image.open(image_stream)
            doc.add_picture(image_stream, width=Inches(3))  # Измените размер, если нужно

        # Добавляем изменённый текст
        doc.add_paragraph(self.decode_encoded_text(self.encoded_text.value))

        word_temp_path = f"/content/updated_{uuid.uuid4()}.docx"
        doc.save(word_temp_path)

        # Конвертируем Word обратно в PDF
        new_pdf_path = f"/content/updated_{uuid.uuid4()}.pdf"
        os.system(f"libreoffice --headless --convert-to pdf {word_temp_path} --outdir /content")

        print(f"Файл сохранён как {new_pdf_path}")

if __name__ == "__main__":
    pdf_url = "/content/final_pdf_with_text (8) (1)_removed.pdf"  # Путь к PDF файлу
    app = PDFEditor(pdf_url)


FileUpload(value={}, accept='.pdf', description='Upload')

Button(description='Внести правки', disabled=True, style=ButtonStyle())

Button(description='Сохранить изменения', disabled=True, style=ButtonStyle())

Button(description='Обновить', style=ButtonStyle())

Text(value='', layout=Layout(width='50%'), placeholder='Введите закодированный текст для поиска')

Button(description='Найти', style=ButtonStyle())

Файл сохранён как /content/updated_0b7deaec-edf3-437b-aaac-24f575f22068.pdf
